# Treinar modelo para identificar entidades

# Tokenizar dados

In [23]:
import zipfile
import pandas as pd
import spacy

In [2]:
dados = []
with zipfile.ZipFile('data/texts.zip','r') as zip:
    for nome in zip.namelist():
        with zip.open(nome) as arquivo:
            conteudo = arquivo.read().decode('utf-8')
            palavras = conteudo.split()
            for palavra in palavras:
                dados.append([nome,palavra])
tabela = pd.DataFrame(dados, columns=['arquivo','palavra'])
tabela

,arquivo,palavra
0,ADI2TJDFT.txt,Órgão
1,ADI2TJDFT.txt,:
2,ADI2TJDFT.txt,Conselho
3,ADI2TJDFT.txt,Especial
4,ADI2TJDFT.txt,Classe
...,...,...
193487,ACORDAOTCU25052016.txt,Ministro-Substituto
193488,ACORDAOTCU25052016.txt,AUGUSTO
193489,ACORDAOTCU25052016.txt,SHERMAN
193490,ACORDAOTCU25052016.txt,CAVALCANTI


In [3]:
tabela.to_csv('data/palavras.csv', index=False,sep='\t')

# ROTULAR DADOS NO FORMATO IOB no sheets

In [4]:
dados_rotulados = pd.read_csv('data/palavras_IOB.tsv',sep='\t')
dados_rotulados

,arquivo,palavra,label
0,TCU4687.txt,GRUPO,O
1,TCU4687.txt,I,O
2,TCU4687.txt,CLASSE,O
3,TCU4687.txt,II,O
4,TCU4687.txt,2ª,B-ORGANIZACAO
...,...,...,...
229272,RR2574407120025020372.txt,de,O
229273,RR2574407120025020372.txt,Chaves,O
229274,RR2574407120025020372.txt,Públicas,O
229275,RR2574407120025020372.txt,Brasileira,O


# Transformar dados para spacy

In [5]:
grupos = dados_rotulados.groupby(by='arquivo')

In [16]:
documentos=[]
for filename in grupos.groups.keys():
    tabela_agrupada = grupos.get_group(filename)[['palavra','label']].values

    conteudo = ''
    anotacoes = {'entities': []}
    inicio = 0
    final = 0

    for texto,label in tabela_agrupada:
        texto = str(texto)
        inicio = final
        final = inicio + len(texto)+1

        if label != 'O':
            anotacao = (inicio,final-1,label)
            anotacoes['entities'].append(anotacao)
    
        conteudo += texto + ' '
    documentos.append((conteudo, anotacoes))

In [17]:
documentos[2]

("EMENTA : AÇÃO DE COBRANÇA - INDENIZAÇÃO DO SEGURO DPVAT - INVALIDEZ PERMANENTE PARCIAL INCOMPLETA - SÚMULA 474/STJ - CORREÇÃO MONETÁRIA - SÚMULA 580 DO STJ - JUROS DE MORA - SÚMULA 426/STJ - HONORÁRIOS ADVOCATÍCIOS - INTERESSE PROCESSUAL - REGISTRO DE OCORRÊNCIA - CERCEAMENTO DE DEFESA - PRESCRIÃO - TERMO INICIAL . `` A indenização do seguro DPVAT , em caso de invalidez parcial permanente do beneficiário , será paga de forma proporcional ao grau da invalidez . '' `` A correção monetária nas indenizações do seguro DPVAT por morte ou invalidez , prevista no § 7º do art . 5º da Lei n . 6.194/1974 , redação dada pela Lei n . 11.482/2007 , incide desde a data do evento danoso . '' `` Os juros de mora na indenização do seguro DPVAT fluem a partir da citação . '' Honorários advocatícios que se mostram adequados para o trabalho técnico realizado , natureza e importância da causa , não desafiam modulação redutiva . Contestada a ação , e produzida prova pericial que permite uma justa resolução

In [18]:
import random

In [24]:
random.shuffle(documentos)
dados_treino = documentos[:int(len(documentos)*0.8)]
dados_validacao = documentos[int(len(documentos)*0.8):]

In [25]:
len(dados_treino)

40

In [27]:
from tqdm import tqdm
from spacy.training import Example

In [28]:
def treinar_modelo(dados_treino,dados_validacao,epochs):
    model = spacy.load('pt_core_news_sm')
    if 'ner' not in model.pipe_names:
        ner = model.create_pipe('ner')
        model.add_pipe(ner,last=True)
    else:
        ner = model.get_pipe('ner')
    for _,anotacoes in dados_treino:
        for ent in anotacoes.get('entities'):
            ner.add_label(ent[2])

    outros_pipelines = [pipeline for pipeline in model.pipe_names if pipeline != 'ner']

    with model.disable_pipes(*outros_pipelines):
        spacy.util.fix_random_seed()
        otimizador = model.create_optimizer()
        for epoch in tqdm(range(epochs), 'Treinando o modelo'):
            random.seed(10)
            random.shuffle(dados_treino)
            losses = {'ner':0.0}

            for textos,anotacoes in dados_treino:
                exemplo = Example.from_dict(model.make_doc(textos), anotacoes)
                model.update([exemplo], drop=0.2, sgd=otimizador, losses=losses)
            print(f"\nEpoch: {epoch} - Loss: {losses['ner']/len(dados_treino)}")

            val_losses = {'ner':0.0}
            exemplos = []
            for textos,anotacoes in dados_validacao:
                exemplo = Example.from_dict(model.make_doc(textos),anotacoes)
                exemplos.append(exemplo)
            for exemplo in exemplos:
                model.update([exemplo],sgd=None,drop=0,losses=val_losses)

            print(f"\nEpoch: {epoch} - Val Loss: {val_losses['ner']/len(dados_validacao)}")
    return model

In [30]:
model = treinar_modelo(dados_treino,dados_validacao,27)

Treinando o modelo:   0%|          | 0/27 [00:00<?, ?it/s]


Epoch: 0 - Loss: 1039.836193271797


Treinando o modelo:   4%|▎         | 1/27 [01:14<32:21, 74.69s/it]


Epoch: 0 - Val Loss: 834.8725585102673

Epoch: 1 - Loss: 698.0798875186547


Treinando o modelo:   7%|▋         | 2/27 [02:26<30:29, 73.19s/it]

...


Epoch: 24 - Val Loss: 21.653428213289573

Epoch: 25 - Loss: 57.57413849568221


Treinando o modelo:  96%|█████████▋| 26/27 [30:31<01:11, 71.81s/it]


Epoch: 25 - Val Loss: 23.498616729347752

Epoch: 26 - Loss: 52.40602450650429


Treinando o modelo: 100%|██████████| 27/27 [31:40<00:00, 70.40s/it]


Epoch: 26 - Val Loss: 22.393006223056688


In [31]:
model.to_disk('modelo')